In [1671]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import os
import numpy as np
import pandas as pd
import wandb

import plotly.express as px

# Pytorch implementation

In [1672]:
use_wandb = True

In [1673]:
config = {
    "lr": 1e-2,
    "dataset": "Titanic",
    "epochs": 50,
    "batch_size": 64,
    "hidden_layers": [32, 16], # [32, 16]
    "dropouts": [0.3], # 0.3
    "scheduler": "", # exponencial / step
    "activations": "ReLU",
    "loss": "binary cross-entropy",
    "optimizer": "Adam",
    "betas": [0.9, 0.99], # [0.9, 0.99]
    "gamma": 0.82 # 0.82
}

In [1674]:
if use_wandb:
    wandb.login()
    wandb.init(project="assignment-1", name="pytorch-32-16", reinit=True, config=config)
    # wandb.config = config

In [1675]:
# device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device = 'cpu'
print(f"Using {device} device")

Using cpu device


## Loading the dataset

In [1676]:
class TitanicDataset(Dataset):
    def __init__(self, data_dir):
        self.labels = pd.read_csv(os.path.join(data_dir, 'labels.csv')).to_numpy(dtype='float32')
        self.data = pd.read_csv(os.path.join(data_dir, 'data.csv')).to_numpy(dtype='float32')

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data = torch.from_numpy(self.data[idx])
        label = torch.from_numpy(self.labels[idx])

        return data, label

In [1677]:
train_data = TitanicDataset(data_dir='data/train/')
val_data = TitanicDataset(data_dir='data/val/')

In [1678]:
features, label = train_data.__getitem__(idx=0)
print(features, label)

tensor([2.0000, 1.0000, 0.3324, 1.0000, 1.0000, 0.5848]) tensor([1.])


In [1679]:
train_dataloader = DataLoader(train_data, batch_size=config['batch_size'], shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=config['batch_size'], shuffle=False)

In [1680]:
for batch, (x, y) in enumerate(train_dataloader):
    print(batch, x.shape, y.shape)
    break

0 torch.Size([64, 6]) torch.Size([64, 1])


## Defining the model

In [1682]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Sequential(
            nn.Linear(6, config['hidden_layers'][0]),
            # nn.Dropout(config['dropouts'][0]),
            nn.ReLU(),
            nn.Linear(config['hidden_layers'][0], config['hidden_layers'][1]),
            nn.Dropout(config['dropouts'][0]),
            nn.ReLU(),
            nn.Linear(config['hidden_layers'][1], 8),
            nn.Dropout(config['dropouts'][0]),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.linear(x)
        return out

In [1683]:
model = NeuralNetwork().to(device)
print(model)

# if use_wandb: 
#     wandb.config['model'] = model.__dict__['_modules']['linear']

NeuralNetwork(
  (linear): Sequential(
    (0): Linear(in_features=6, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): Dropout(p=0.3, inplace=False)
    (4): ReLU()
    (5): Linear(in_features=16, out_features=8, bias=True)
    (6): Dropout(p=0.3, inplace=False)
    (7): ReLU()
    (8): Linear(in_features=8, out_features=1, bias=True)
    (9): Sigmoid()
  )
)


## Training

In [1684]:
loss_fn = nn.BCELoss()
if config['optimizer'] == 'Adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], betas=[config['betas'][0], config['betas'][1]])
elif config['optimizer'] == 'SGD':
    optimizer = torch.optim.SGD(model.parameters(), lr=config['lr'])
elif config['optimizer'] == 'RMSprop':
    optimizer = torch.optim.RMSprop(model.parameters(), lr=config['lr'])

In [1685]:
# if use_wandb:
#     wandb.watch(model, loss_fn, log='all', log_freq=10)

In [1686]:
def calculate_acc(y_pred, y) -> float:
    y_pred = (y_pred>0.5)
    
    return ((y == y_pred).sum().item())/len(y)

In [1687]:
def train(dataloader, model, loss_fn, optimizer, scheduler, epoch):
    model.train()
    avg_loss, avg_acc = 0, 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)

        # Get prediction and compute loss
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        avg_loss += loss.item()

        # Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_acc += calculate_acc(y_pred, y)

    # Change lr based on the provided scheduler
    if epoch > 9:
        scheduler.step()
    
    avg_loss /= len(dataloader)
    avg_acc /= len(dataloader)
        
    return (avg_loss, avg_acc)

In [1688]:
def val(dataloader, model, loss_fn, epoch):    
    model.eval()
    avg_loss, avg_acc = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            avg_loss += loss_fn(y_pred, y).item()
            avg_acc += calculate_acc(y_pred, y)
    
    avg_loss /= len(dataloader)
    avg_acc /= len(dataloader)
    
    return (avg_loss, avg_acc)

In [1689]:
epochs = config['epochs']
train_history = {'loss': [], 'acc': []}
val_history = {'loss': [], 'acc': []}
best_epoch = {}

if config['scheduler'] == 'exponencial':
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=config['gamma'])
elif config['scheduler'] == 'step':
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    
# scheduler = torch.optim.lr_scheduler.PolynomialLR(optimizer)
for epoch in range(epochs):
    train_loss, train_acc = train(train_dataloader, model, loss_fn, optimizer, scheduler, epoch)
    val_loss, val_acc = val(val_dataloader, model, loss_fn, epoch)

    train_history['loss'].append(train_loss)
    train_history['acc'].append(train_acc)

    val_history['loss'].append(val_loss)
    val_history['acc'].append(val_acc)
    
    if (epoch % 10 == 0):
        print(f"Epoch {epoch}")
        print(f'loss: {train_loss:>5f} acc: {train_acc:>5f}')
        print(f'val loss: {val_loss:>5f} val acc: {val_acc:>5f}')
        print(f'lr: {optimizer.param_groups[0]["lr"]:>5f}')
        print('-------------------------------')

    if use_wandb:
        wandb.log({'epoch': epoch, 'loss': train_loss, 'accuracy': train_acc, 'val_loss':val_loss, 'val_accuracy': val_acc, 'lr': optimizer.param_groups[0]["lr"]})

Epoch 0
loss: 0.678636 acc: 0.565168
val loss: 0.646431 val acc: 0.630208
lr: 0.010000
-------------------------------
Epoch 10
loss: 0.418959 acc: 0.827172
val loss: 0.427621 val acc: 0.837153
lr: 0.010000
-------------------------------
Epoch 20
loss: 0.385344 acc: 0.842111
val loss: 0.425885 val acc: 0.840278
lr: 0.010000
-------------------------------
Epoch 30
loss: 0.388041 acc: 0.854116
val loss: 0.424665 val acc: 0.840278
lr: 0.010000
-------------------------------
Epoch 40
loss: 0.346833 acc: 0.834108
val loss: 0.451773 val acc: 0.830903
lr: 0.010000
-------------------------------


In [ ]:
print(f'Results after {config["epochs"]} epochs for a model with:\n{config["optimizer"]} optimizer, {config["lr"]} lr, {config["batch_size"]} batch size, {config["loss"]} and with the following architecture:')
print(model)

if use_wandb:
    wandb.finish()

In [1691]:
fig = px.line({'loss': train_history['loss'], 'val_loss': val_history['loss']})
fig.show()

In [1692]:
fig = px.line({'acc': train_history['acc'], 'val_acc': val_history['acc']})
fig.show()